# Dreambooth inpainting finetuning

Finetune Stable Diffusion inpainting model on custom images. Replace custom mask in any image with finetuned object.

1. **Fine-tuning Stable Diffusion inpainting model on custom images:** Stable Diffusion is a state-of-the-art machine learning model, used for the purpose of image inpainting. Inpainting is a process that involves filling in the missing part of any image using the existing data. In this snippet, the Stable Diffusion model is being fine-tuned on custom images.

2. **Replace mask in any image with the fine-tuned object:** After the fine-tuning process, the subsequent task is to use the fine-tuned model to replace the mask in any image with the fine-tuned object. This essentially means replacing a part of any image, marked by a mask, with the object generated by the model.

In [ ]:
!pip install -U diffusers transformers ftfy gradio accelerate

In [ ]:
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121

In [ ]:
!wget https://raw.githubusercontent.com/huggingface/diffusers/main/examples/research_projects/dreambooth_inpaint/train_dreambooth_inpaint.py

In [ ]:
import inspect
from typing import List, Optional, Union
import numpy as np
import torch
import os
import PIL
import gradio as gr
from diffusers import StableDiffusionInpaintPipeline
import requests
from io import BytesIO

In [ ]:
def image_grid(imgs, rows, cols, resize=256):
    if resize is not None:
        imgs = [img.resize((resize, resize)) for img in imgs]
    w, h = imgs[0].size
    grid = PIL.Image.new("RGB", size=(cols * w, rows * h))
    grid_w, grid_h = grid.size

    for i, img in enumerate(imgs):
        grid.paste(img, box=(i % cols * w, i // cols * h))
    return grid

In [ ]:
device = "cuda"
model_path = "stability-ai/stable-diffusion-inpainting-2"

pipe = StableDiffusionInpaintPipeline.from_pretrained(
    model_path,
    torch_dtype=torch.float16,
).to(device)

In [ ]:
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
!accelerate launch train_dreambooth_inpaint.py \
    --pretrained_model_name_or_path="stabilityai/stable-diffusion-2-inpainting"  \
    --instance_data_dir="images/dog" \
    --output_dir="stable-diffusion-inpainting-painting" \
    --instance_prompt="old painting" \
    --resolution=512 \
    --mixed_precision="fp16" \
    --train_batch_size=1 \
    --learning_rate=5e-6 \
    --lr_scheduler="constant" \
    --lr_warmup_steps=0 \
    --max_train_steps=500 \
    --gradient_accumulation_steps=2 \
    --gradient_checkpointing \
    --train_text_encoder \
    --seed="0"
    --push_to_hub